In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
# load data
rna_path = "~/Documents/Single cell/MA_rm_batch_effect/data set/dna_atac/dta_rna.csv"
atac_path = "~/Documents/Single cell/MA_rm_batch_effect/data set/dna_atac/dta_atac.csv"
dta_rna = pd.read_csv(rna_path, index_col=0)
dta_atac = pd.read_csv(atac_path, index_col=0)
dta = pd.concat([dta_rna, dta_atac], axis = 1)
dta = dta.T

In [ ]:
adata = sc.AnnData(X = dta)
adata.var_names_make_unique()
sc.pp.filter_cells(adata, min_genes=10)
sc.pp.filter_genes(adata, min_cells=1)
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, inplace=True)
adata

In [ ]:
# np.mean(adata.obs.n_genes_by_counts)
# np.std(adata.obs.n_genes_by_counts)
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 10, :]


In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
adata.raw = adata
adata = adata[:, adata.var.highly_variable]

In [ ]:
sc.tl.pca(adata, svd_solver='arpack', n_comps = 50, copy = True)